Faire un programme qui:

- Trouve les 10 villes de France les + peuplées (e.g https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées)
- Pour chaque ville, trouve sa distance avec les autres.
  - par exemple, avec google maps api (nécessite de s'authentifier / créer un token) avec https://github.com/googlemaps/google-maps-services-python (cf. doc token dans le readme)
  - ou avec https://fr.distance24.org/ (pas besoin de s'authentifier)
- Trouve les villes les plus proches

In [1]:
import requests
from bs4 import BeautifulSoup


def get_soup_from_url(url):
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')

    return soup


def get_biggest_cities(limit=10):
    url = 'https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées'

    soup = get_soup_from_url(url)
    cities = []

    table = soup.find('tbody')
    for row in table.find_all('tr')[1:11]:
        cells = row.find_all('td')
        city = cells[1].find('a').text.strip()
        cities.append(city)

    return cities[:limit]

In [2]:
def get_distance_between(city_to, city_from):
        query = "https://fr.distance24.org/route.json?stops={}|{}".format(city_to, city_from)
        
        data = requests.get(query).json()

        return data['distance']

In [3]:
cities = get_biggest_cities()

for idx, c1 in enumerate(cities):
    for c2 in cities[idx+1:]:
        print("{} => {} = {}".format(c1, c2, get_distance_between(c1, c2)))

Paris => Marseille = 661
Paris => Lyon = 392
Paris => Toulouse = 589
Paris => Nice = 688
Paris => Nantes = 343
Paris => Montpellier = 596
Paris => Strasbourg = 397
Paris => Bordeaux = 500
Paris => Lille = 204
Marseille => Lyon = 278
Marseille => Toulouse = 319
Marseille => Nice = 159
Marseille => Nantes = 696
Marseille => Montpellier = 126
Marseille => Strasbourg = 617
Marseille => Bordeaux = 506
Marseille => Lille = 835
Lyon => Toulouse = 360
Lyon => Nice = 300
Lyon => Nantes = 516
Lyon => Montpellier = 251
Lyon => Strasbourg = 384
Lyon => Bordeaux = 436
Lyon => Lille = 558
Toulouse => Nice = 469
Toulouse => Nantes = 466
Toulouse => Montpellier = 196
Toulouse => Strasbourg = 737
Toulouse => Bordeaux = 212
Toulouse => Lille = 792
Nice => Nantes = 792
Nice => Montpellier = 273
Nice => Strasbourg = 545
Nice => Bordeaux = 638
Nice => Lille = 835
Nantes => Montpellier = 584
Nantes => Strasbourg = 710
Nantes => Bordeaux = 276
Nantes => Lille = 508
Montpellier => Strasbourg = 629
Montpellier

In [4]:
import itertools

combinations = list(itertools.combinations(cities, 2))

distances = []
for origin, dest in combinations:
    distances.append(get_distance_between(origin, dest))

In [9]:
import pandas as pd
df = pd.DataFrame(combinations, columns=['origin', 'dest'])
df['distance'] = distances

df.head(10)

,origin,dest,distance
0,Paris,Marseille,661
1,Paris,Lyon,392
2,Paris,Toulouse,589
3,Paris,Nice,688
4,Paris,Nantes,343
5,Paris,Montpellier,596
6,Paris,Strasbourg,397
7,Paris,Bordeaux,500
8,Paris,Lille,204
9,Marseille,Lyon,278


In [8]:
df.sort_values('distance').head(10)

,origin,dest,distance
13,Marseille,Montpellier,126
11,Marseille,Nice,159
26,Toulouse,Montpellier,196
8,Paris,Lille,204
28,Toulouse,Bordeaux,212
20,Lyon,Montpellier,251
31,Nice,Montpellier,273
37,Nantes,Bordeaux,276
9,Marseille,Lyon,278
18,Lyon,Nice,300
